In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

import rtbm.layers as layers
import rtbm.model as mdl

import warnings
warnings.filterwarnings('ignore')

from rtbm.costfunctions import mse
from rtbm import minimizer

from rtbm.mathtools import theta_1d, logtheta_1d_phaseI
from rtbm.riemann_theta.riemann_theta import RiemannTheta

import rtbm.layers as layers
import rtbm.model as mdl

import mpmath
from numpy import frompyfunc

The history saving thread hit an unexpected error (DatabaseError('database disk image is malformed',)).History will not be written to the database.


Process PoolWorker-12:
Process PoolWorker-4:
Traceback (most recent call last):
Traceback (most recent call last):
Process PoolWorker-16:
Process PoolWorker-3:
Process PoolWorker-15:
Process PoolWorker-5:
Process PoolWorker-6:
Process PoolWorker-10:
Process PoolWorker-1:
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Process PoolWorker-13:
Traceback (most recent call last):
Process PoolWorker-9:
    self.run()
    self.run()
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
Process PoolWorker-8:
Process PoolWorker-7:
Process PoolWorker-2:
Traceback (most recent call last):
  File "/usr/lib/python2.7/multiprocessing/process.py", line 258, in _bootstrap
Process PoolWorker-14:
Traceback (most recent call last):
Traceback (most recent call last):
Process PoolWorker-11:
  File "/usr/lib/python2.7/multiprocessin

# Derivatives test

In [12]:
Q=np.zeros((1,1), dtype=complex)
V=np.zeros((1,1), dtype=complex)
V[0,0]=0.1*2j*np.pi
Q[0,0]=0.01*2*np.pi

#print(theta_1d(V,Q,0))


print(RiemannTheta(V/(2j*np.pi),1j*Q/(2*np.pi), derivs=[[1],[1]]))

#print(RiemannTheta(V/(2j*np.pi),1j*Q/(2*np.pi), derivs=[[1],[1]]))

#mpmath.jtheta(3,V[0,0],Q[0,0])

(1434.49605524+5.69872548808e-14j)


In [13]:
X=np.random.uniform(-50, 50,(1000,1))

In [14]:
%%time
t = RiemannTheta(X/(2j*np.pi),1j*Q/(2*np.pi))

CPU times: user 100 ms, sys: 0 ns, total: 100 ms
Wall time: 23.1 ms


In [15]:
test= frompyfunc(lambda z, q: mpmath.jtheta(3, z, q), 2, 1)

In [16]:
%%time
t = test(np.pi*X/(2j*np.pi),np.exp(1j*np.pi*1j*Q[0,0]/(2*np.pi)))

CPU times: user 9.51 s, sys: 2.13 s, total: 11.6 s
Wall time: 9.59 s


In [17]:
Q=np.zeros((1,1), dtype=float)
V=np.zeros((1,1), dtype=float)
V[0,0]=0.1
Q[0,0]=0.01


print(theta_1d(V,Q,0))
print(np.exp(logtheta_1d_phaseI(V,Q,0)))

print(RiemannTheta(V/(2j*np.pi),-1*Q/(2j*np.pi)))

(41.3273135097+0j)
[41.32731351+0.j]
(41.3273135097+0j)


In [18]:
41.0/25.0

1.64

# Linear layer test

In [19]:
def funcA(x):
    return 0.6-0.3*x

def funcB(x):
    return -0.5+0.8*x


def func(x1,x2):
    return 0.6-0.3*x1+1.2*x2

X1 = np.linspace(-5.3, 5, 1000)
X2 = np.linspace(-5.5, 5, 1000)

X = np.stack((X1,X2))

#Y = func(X1,X2).reshape((1,X.shape[1]))
Y = np.stack((funcA(X1),funcB(X2)))

M = mdl.Model()
M.add(layers.Linear(2,2))

#minim = minimizer.CMA(True)
#minim.train(mse(), M, X, Y, tolfun=1e-3)

M = mdl.Model()
M.add(layers.Linear(2,1))
M.add(layers.Linear(1,2))


minim = minimizer.SGD()
minim.train(mse(), M, X, Y, lr=0.0001,maxiter=1000)

Iteration 0 in 0.00(s), cost = 3.007016
Iteration 100 in 0.03(s), cost = 2.729608
Iteration 200 in 0.06(s), cost = 2.512413
Iteration 300 in 0.09(s), cost = 2.331413
Iteration 400 in 0.12(s), cost = 2.171677
Iteration 500 in 0.16(s), cost = 2.023488
Iteration 600 in 0.19(s), cost = 1.880449
Iteration 700 in 0.22(s), cost = 1.738548
Iteration 800 in 0.25(s), cost = 1.595703
Iteration 900 in 0.28(s), cost = 1.451519
('Cost: ', 1.3085300331987575)
('Sol: ', array([-0.34664693, 0.01670776, 0.74753964, -0.69191109, -0.69506442,
       -0.05901294, 0.30909227]))
Time: 0 s


array([-0.34664693, 0.01670776, 0.74753964, -0.69191109, -0.69506442,
       -0.05901294, 0.30909227])

# E(h|v) SGD test

In [ ]:
def func(x):
    return np.sin(x)+x

X = np.linspace(-5, 5, 1000)
X = X.reshape((1,X.shape[0]))

Y = func(X[:,None]).reshape((1,X.shape[1]))

M = mdl.Model()
M.add(layers.DiagExpectationUnitLayer(1,1, phase=1))

minim = minimizer.SGD()

minim.train(mse(), M, X, Y, lr=0.01,maxiter=1000,noise=0.01)

plt.plot(X.flatten(), Y.flatten(),"og-", label='fit')
plt.plot(X.flatten(), np.real(M.predict(X)).flatten(),"ob-", label='fit')

Iteration 0 in 0.11(s), cost = 3.164177
Iteration 100 in 8.05(s), cost = 0.326603
Iteration 200 in 15.94(s), cost = 0.283272
Iteration 300 in 23.89(s), cost = 0.274579
Iteration 400 in 32.73(s), cost = 0.263310


In [ ]:
M = mdl.Model()
M.add(layers.DiagExpectationUnitLayer(1,3,phase=1j))
M.add(layers.DiagExpectationUnitLayer(2,1,phase=1j))

minim = minimizer.CMA(True)

minim.train(mse(), M, X, Y, maxiter=1000)
plt.plot(X.flatten(), Y.flatten(),"og-", label='fit')
plt.plot(X.flatten(), np.real(M.predict(X)).flatten(),"ob-", label='fit')

CMA on 16 cpu(s) enabled
(5_w,10)-aCMA-ES (mu_w=3.2,w_1=45%) in dimension 10 (seed=791460, Sat Nov  4 01:23:01 2017)
Iterat #Fevals   function value  axis ratio  sigma  min&max std  t[m:s]
    1     10 3.954760253979186e+00 1.0e+00 9.78e-01  9e-01  1e+00 0:00.4
    2     20 3.747624173810217e+00 1.2e+00 9.71e-01  9e-01  1e+00 0:00.7
    3     30 3.348135026821898e+00 1.3e+00 9.98e-01  9e-01  1e+00 0:01.0
   12    120 3.504810576348552e+00 1.8e+00 6.62e-01  5e-01  7e-01 0:04.0
   24    240 3.138099658848774e+00 2.6e+00 5.57e-01  3e-01  7e-01 0:08.3
   39    390 2.083672916065300e+00 5.4e+00 4.79e-01  2e-01  7e-01 0:13.4
   55    550 3.754377078249441e-01 7.3e+00 2.63e-01  7e-02  4e-01 0:19.5
   76    760 2.049896798754062e-01 9.1e+00 2.14e-01  6e-02  3e-01 0:26.8
  100   1000 1.473206550078088e-01 1.2e+01 1.43e-01  3e-02  2e-01 0:35.0
  127   1270 8.137108463624768e-02 3.8e+01 1.75e-01  2e-02  3e-01 0:44.2
  157   1570 3.220958828479951e-02 6.0e+01 1.97e-01  2e-02  4e-01 0:54.6
  190   

In [ ]:
def func(x1,x2):
    return 0.2*x1+0.4*x2+0.8

X1 = np.linspace(-5.3, 5, 1000)
X2 = np.linspace(-5.5, 5, 1000)

X = np.stack((X1,X2))

Y = func(X1[:,None],X2[:,None]).reshape((1,X1.shape[0]))

M = mdl.Model()
M.add(layers.DiagExpectationUnitLayer(2,1))
M.add(layers.DiagExpectationUnitLayer(1,1))


minim = minimizer.SGD()
minim.train(mse(), M, X, Y, lr=0.1,maxiter=400)

#plt.plot(X.flatten(), Y.flatten(),"og-", label='fit')
#plt.plot(X.flatten(), np.real(M.predict(X)).flatten(),"ob-", label='fit')

# Layer test

In [ ]:
M = mdl.Model()
M.add(layers.DiagExpectationUnitLayer(1,3))
M.add(layers.DiagExpectationUnitLayer(3,1))

In [ ]:
def func(x):
    return np.sin(x)

X = np.linspace(0, 10, 5)
X = X.reshape((1,X.shape[0]))

Y = func(X[:,None]).reshape((1,X.shape[1]))

In [ ]:
plt.plot(X.flatten(), Y.flatten(),"ob-")

In [ ]:
minim = minimizer.CMA(True)
minim.train(mse(), M, X, Y, tolfun=1e-3)

In [ ]:
npoints = 5
test_X = (np.linspace(0, 10, npoints)).reshape((1, npoints))

plt.plot(X.flatten(), Y.flatten(),"og-", label='target')
plt.plot(test_X.flatten(), np.real(M.predict(test_X)).flatten(),"ob-", label='fit')
plt.legend()

# Misc tests

In [ ]:
L = layers.MaxPosLayer(3,1)

In [ ]:
L.feedin(np.array([Y,2*Y,1*Y]).reshape(3,5))

In [ ]:
np.array([Y,2*Y,Y]).reshape(3,5)

In [ ]:
np.empty(0)

In [ ]:
derivative_1d_theta_phaseI([0.1],[1j],0)